<a href="https://colab.research.google.com/github/Shashank-Shukla/PyTorch-practice/blob/master/ResNet_using_Torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn

In [ ]:
'''
First create ResNet blocks
Then later we'll build the ResNet architecture
'''

class block(nn.Module):
  # params = input_channel_size, output_channel_dims, 
  # initialization function to predefine var and processes
  def __init__(self, in_channels, out_channels, identity_downsample=None ,stride=1):
    super(block, self).__init__()
    # channel expands after it is processed by each block [as in research paper]
    self.expansion = 4    # channel expands 4x [from 64 to 256]
    # Now for ResNet50 conv2_x, this layer is a {1x1,64} layer
    self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
    '''
    stride controls the stride for the cross-correlation,
    padding controls the amount of implicit zero-paddings on both sides,
    param(kernel_size,stride,padding) = either 'int' [if Height == Width] 
    or tuple(int,int) [first int is Height and 2nd is Width]
    '''
    # Now between the next Conv layer and this Conv layer we'll have a batch norm
    self.bn1 = nn.BatchNorm2d(out_channels)
    # After the first layer, next will have a dim of: {3x3,64}
    self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
    self.bn2 = nn.BatchNorm2d(out_channels)
    # The third layer will have a dim. of: {1x1,256}
    self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
    self.bn3 = nn.BatchNorm2d(out_channels*self.expansion)
    self.relu = nn.ReLU()
    self.identity_downsample = identity_downsample
    # Identity mapping to the conv layer so it's the same shape after processing
    self.stride = stride

  def forward(self, x):
    identity = x.clone()

    # Start computation:
    # Conv -> bn -> ReLU -> Conv -> bn -> Relu -> Conv -> bn
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.conv2(x)
    x = self.bn2(x)
    x = self.relu(x)
    x = self.conv3(x)
    x = self.bn3(x)

    # Now we use the identity block or skip-connection
    # We use this block to change the shape
    if self.identity_downsample is not None:
      identity = self.identity_downsample(identity)
    
    # Now we'll add this skip-connection to the output of Conv layers stack
    x += identity
    # Now we use the ReLU activation over this whole stack
    x = self.relu(x)
    
    return x


In [ ]:
class ResNet(nn.Module):
  def __init__(self, block, layers, img_channels, no_classes):
    '''
    Inheriting the Block class
    layers is a tuple that defines how many instances of the block we need to create
                [3, 4, 6, 3]   ..... stride of each: [1, 2, 2, 2]
    img_channels is no. of image channels [for RGB image: 3]
    no_classes is the number of classes we have in dataset [cat, dog, car...etc]
    '''
    super(ResNet, self).__init__()
    
    # Now let's initialize the modules
    self.in_channels = 64
    self.conv1 = nn.Conv2d(img_channels, out_channels=64, kernel_size=7, stride=2, padding=3)
    self.bn1 = nn.BatchNorm2d(64)
    self.relu = nn.ReLU()
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    # Define ResNet layers
    self.layer1 = self.create_layer(block, layers[0], out_channels=64, stride=1)
    self.layer2 = self.create_layer(block, layers[1], out_channels=128, stride=2)
    self.layer3 = self.create_layer(block, layers[2], out_channels=256, stride=2)
    self.layer4 = self.create_layer(block, layers[3], out_channels=512, stride=2)

    self.avgpool = nn.AdaptiveAvgPool2d((1,1))
    self.fc = nn.Linear(2048, no_classes)   # o/p: 512*4 = 2048

  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)

    x = self.avgpool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc(x)
    return x

  def create_layer(self, block, no_res_blocks, out_channels, stride):
    identity_downsample = None
    layers = []
    '''
    We need to add the identity whenever we change conv2_x layer stack and we 
    determine that if stride changes to 1 or if input channel dimension is not
    same as output channel dimension*4
    '''
    if stride != 1 or self.in_channels != out_channels*4:
      identity_downsample = nn.Sequential(nn.Conv2d(self.in_channels, out_channels*4, kernel_size=1, stride=stride),
                                          nn.BatchNorm2d(out_channels*4))
    layers.append(block(self.in_channels, out_channels, identity_downsample, stride))
    # new input = prev-output*expansion
    self.in_channels = out_channels*4   # new-input = 256

    # Now it's time to loop through and recall blocks for computation
    for i in range(no_res_blocks-1):
      # -1 coz we already computed a block on line:28 above
      layers.append(block(self.in_channels, out_channels))
      '''
      So input will be 256 channels, which will be mapped to output 64 channels 
      and then reproduce an output of 64*4=256 again in the end
      '''

    return nn.Sequential(*layers)


In [ ]:
# Implementing ResNet50
def ResNet50(img_channels=3, no_classes=1000):
  return ResNet(block, [3, 4, 6, 3], img_channels, no_classes)

In [ ]:
# Implementing ResNet101
def ResNet101(img_channels=3, no_classes=1000):
  return ResNet(block, [3, 4, 23, 3], img_channels, no_classes)

In [ ]:
# Implementing ResNet152
def ResNet152(img_channels=3, no_classes=1000):
  return ResNet(block, [3, 8, 36, 3], img_channels, no_classes)

In [ ]:
# Testing these models:
from time import time as t
def test():
  net = ResNet50()
  x = torch.randn(16, 3, 224, 224)
  print(x.shape)
  print(x)
  t1s = t()
  y = net(x).to('cuda')
  t2s = t()
  tdiff1 = t2s-t1s
  t1s = t()
  y = net(x).to('cuda')
  t2s = t()
  tdiff2 = t2s-t1s
  t1s = t()
  y = net(x).to('cuda')
  t2s = t()
  tdiff3 = t2s-t1s
  #y = net(x)
  print(y.shape)
  print("ResNet50: ",tdiff1,"s")
  print("ResNet101: ",tdiff2,"s")
  print("ResNet152: ",tdiff3,"s")

In [ ]:
test()

In [ ]:
'''
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())
print(torch.cuda.current_device())
'''

0
1
Tesla T4
True
0
0
Tesla T4


In [ ]:
'''import glob
from PIL import Image
from torchvision import transform
import matplotlib.pyplot as plt
%matplotlib inline
imms = glob.glob('*JPG')
print(imms)
for i in imms:
  immg = Image.open(i)
  trans = transform.ToPILImage()
  trans1 = transform.ToTensor()
  plt.imshow(trans(trans1(immg)))'''

ImportError: ignored

In [ ]:
ls

first.JPG  sample_data/  second.JPG  third.JPG
